# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
  

- 🤝 Breakout Room #2:
  1. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas)
  2. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

We'll also get the "star of the show" today, which is Ragas!

In [2]:
!pip install -qU ragas

As well, instead of the remote hosted solution that we used last week (Pinecone), we'll be leveraging Meta's [FAISS](https://github.com/facebookresearch/faiss) as the backend for our LangChain `VectorStore`.

We'll also install `unstructured` (from [Unstructured-IO](https://github.com/Unstructured-IO/unstructured)) and its dependencies which will allow us to load PDFs using the `UnstructuredPDFLoader` in the `langchain-community` package!

In [3]:
!pip install -qU faiss_cpu pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [4]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.1.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [5]:
!git clone https://github.com/AI-Maker-Space/DataRepository

fatal: destination path 'DataRepository' already exists and is not an empty directory.


In [6]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "DataRepository/MuskComplaint.pdf",
)

documents = loader.load()

In [7]:
documents[0].metadata

{'source': 'DataRepository/MuskComplaint.pdf',
 'file_path': 'DataRepository/MuskComplaint.pdf',
 'page': 0,
 'total_pages': 46,
 'format': 'PDF 1.7',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': '',
 'creationDate': '',
 'modDate': '',
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [9]:
len(documents)

159

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [10]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a FAISS VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [11]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

####❓ Question #1:

List out a few of the techniques that FAISS uses that make it performant.

> NOTE: Check the [repository](https://github.com/facebookresearch/faiss) for more information about FAISS!

> Answer:
> FAISS solves this issue by providing efficient algorithms for similarity search and clustering that are capable of dealing with large-scale, high-dimensional data.
>
> FAISS has numerous indexing structures that can be utilised to speed up the search, including LSH, IVF, and PQ. It also includes GPU support, which enables further search acceleration. FAISS also offers an 
> estimated nearest neighbour search, which delivers approximate nearest neighbours with a quality guarantee.



#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [12]:
retriever = vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [13]:
retrieved_documents = retriever.invoke("Who is the plantiff?")

In [14]:
for doc in retrieved_documents:
  print(doc)

page_content='would be owned by the foundation and used ‘for the good of the world’[.]” Plaintiff \nreplied: “Agree on all.” Ex. 2 at 1.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 27, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='property and derivative works funded by those monies, Plaintiff is presently unable to ascertain his \ninterest in or the use, allocation, or distribution of assets without an accounting. Plaintiff is therefore \nentitled to an accounting.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 32, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='1

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [17]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [18]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [19]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [20]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

Let's test it out!

In [21]:
question = "Who is the plantiff?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Elon Musk


In [22]:
question = "What does this complaint pertain to?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

The complaint pertains to breach of fiduciary duty, unfair business practices, and accounting.
[Document(page_content='1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \n18 \n19 \n20 \n21 \n22 \n23 \n24 \n25 \n26 \n27 \n28 \n \n \n– 31 – \nCOMPLAINT \n \nTHIRD CAUSE OF ACTION \nBreach of Fiduciary Duty  \nAgainst All Defendants \n133. \nPlaintiff realleges and incorporates by reference only paragraphs of this Complaint \nnecessary for his claim of Breach of Fiduciary Duty. \n134. \nUnder California law, Defendants owe fiduciary duties to Plaintiff, including a duty \nto use Plaintiff’s contributions for the purposes for which they were made. E.g., Cal. Bus. & Prof. \nCode § 17510.8. Defendants have repeatedly breached their fiduciary duties to Plaintiff, including \nby:', metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 30, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

# 🤝 Breakout Room #2

## Task 1: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evluating on every core metric today, but in order to do that - we'll need to creat a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

> NOTE: This process will use `gpt-3.5-turbo-16k` as the base generator and `gpt-4` as the critic - if you're attempting to create a lot of samples please be aware of cost, as well as rate limits.

In [23]:
eval_documents = documents

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 400
)

eval_documents = text_splitter.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

> Answer:  to prevent creating the test data using the same chunks that are also in the vector store

In [24]:
len(documents)

159

In [25]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.with_openai()

testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.25, reasoning: 0.25, multi_context: 0.5})

embedding nodes:   0%|          | 0/318 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/10 [00:00<?, ?it/s]

####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

> Answer: 
> Language Language Models (LLMs) possess the capability to transform simple questions into more complex ones effectively. To generate medium to hard samples from the provided documents, we employ > the following methods:
>
> Reasoning: Rewrite the question in a way that enhances the need for reasoning to answer it effectively.
>
> Conditioning: Modify the question to introduce a conditional element, which adds complexity to the question.
>
> Multi-Context: Rephrase the question in a manner that necessitates information from multiple related sections or chunks to formulate an answer.

Let's look at the output and see what we can learn about it!

In [26]:
testset.test_data[0]

DataRow(question='What strategy video game did OpenAI compete in?', contexts=['77. \nInitial work at OpenAI followed much in the footsteps of DeepMind. OpenAI used \nreinforcement learning to play a game. Instead of playing chess, however, OpenAI competed in \nDota 2, a strategy video game with far more moving pieces than chess. OpenAI’s team quickly'], ground_truth='OpenAI competed in Dota 2, a strategy video game.', evolution_type='simple')

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [27]:
test_df = testset.to_pandas()

In [28]:
test_df

,question,contexts,ground_truth,evolution_type,episode_done
0,What strategy video game did OpenAI compete in?,[77. \nInitial work at OpenAI followed much in...,"OpenAI competed in Dota 2, a strategy video game.",simple,True
1,How did OpenAI use reinforcement learning in t...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to play Dot...,simple,True
2,How would IP assets be used to generate profit...,"[business model were valid, it would radically...",nan,reasoning,True
3,Which strategy video game did OpenAI beat a wo...,[77. \nInitial work at OpenAI followed much in...,OpenAI beat a world-champion program in the ga...,reasoning,True
4,"""What was the impact of OpenAI's models on fut...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,Their publication did prove to be useful to th...,multi_context,True
5,"""What game did OpenAI use reinforcement learni...",[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to compete ...,multi_context,True
6,What was Mr. Page's response to Mr. Musk's con...,"[Page, then-CEO of Google’s parent company Alp...",Mr. Page responded that would merely “be the n...,multi_context,True
7,What are the US officials' concerns about UAE'...,[chip fabrication plants. If $10 billion from ...,US officials are concerned about UAE's ties to...,multi_context,True
8,What were Stephen Hawking's concerns about AGI...,[18. \nMr. Musk has long recognized that AGI p...,Stephen Hawking's concerns about AGI in the wr...,multi_context,True
9,What model did OpenAI release in January 2018 ...,[those connections to the target language. \n7...,OpenAI released the Generative Pre-Trained Tra...,reasoning,True


In [29]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [30]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [31]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [75]:
response_dataset[8]

{'question': "What were Stephen Hawking's concerns about AGI in the wrong hands?",
 'answer': 'Stephen Hawking\'s concerns about AGI in the wrong hands were that "the future doesn\'t need us."',
 'contexts': ['18. \nMr. Musk has long recognized that AGI poses a grave threat to humanity—perhaps \nthe greatest existential threat we face today. His concerns mirrored those raised before him by \nluminaries like Stephen Hawking and Sun Microsystems founder Bill Joy. Our entire economy is \nbased around the fact that humans work together and come up with the best solutions to a hard task. \nIf a machine can solve nearly any task better than we can, that machine becomes more economically \nuseful than we are. As Mr. Joy warned, with strong AGI, “the future doesn’t need us.” Mr. Musk',
  'to its mission to develop AGI for the benefit of humanity, not to personally benefit the individual \nDefendants and the largest technology company in the world.  \nDetailed Allegations \nA. \nMr. Musk’s Conc

## Task 2: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [35]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [36]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [35]:
results

{'faithfulness': 0.9722, 'answer_relevancy': 0.9322, 'context_recall': 1.0000, 'context_precision': 0.9833, 'answer_correctness': 0.7670}

In [37]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What strategy video game did OpenAI compete in?,Dota 2,[77. \nInitial work at OpenAI followed much in...,"OpenAI competed in Dota 2, a strategy video game.",1.000000,0.953850,1.0,1.000000,0.716374
1,How did OpenAI use reinforcement learning in t...,OpenAI used reinforcement learning to compete ...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to play Dot...,1.000000,0.971244,1.0,1.000000,0.746960
2,How would IP assets be used to generate profit...,IP assets would be slid into a new for-profit ...,"[business model were valid, it would radically...",nan,1.000000,0.925542,0.5,0.000000,0.181010
3,Which strategy video game did OpenAI beat a wo...,Dota 2,[77. \nInitial work at OpenAI followed much in...,OpenAI beat a world-champion program in the ga...,1.000000,0.913615,1.0,1.000000,0.963256
4,"""What was the impact of OpenAI's models on fut...",The impact of OpenAI's models on future model ...,"[challenging.” At the time, OpenAI stated that...",Their publication did prove to be useful to th...,1.000000,0.941153,1.0,1.000000,0.839459
5,"""What game did OpenAI use reinforcement learni...",Dota 2,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to compete ...,1.000000,0.909523,1.0,1.000000,0.964963
6,What was Mr. Page's response to Mr. Musk's con...,Mr. Page responded that the potential replacem...,"[Page, then-CEO of Google’s parent company Alp...",Mr. Page responded that would merely “be the n...,1.000000,0.917549,1.0,0.916667,0.596798
7,What are the US officials' concerns about UAE'...,The US officials are concerned about the UAE's...,[chip fabrication plants. If $10 billion from ...,US officials are concerned about UAE's ties to...,0.666667,0.970784,1.0,0.833333,0.745019
8,What were Stephen Hawking's concerns about AGI...,Stephen Hawking's concerns about AGI in the wr...,[18. \nMr. Musk has long recognized that AGI p...,Stephen Hawking's concerns about AGI in the wr...,1.000000,0.964590,0.0,1.000000,0.724574
9,What model did OpenAI release in January 2018 ...,OpenAI released the Generative Pre-Trained Tra...,[those connections to the target language. \n7...,OpenAI released the Generative Pre-Trained Tra...,1.000000,0.887711,1.0,0.833333,0.847443


## Task 3: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

In [38]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [39]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [40]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [41]:
response = retrieval_chain.invoke({"input": "Who is the plantiff?"})

In [42]:
print(response["answer"])

The plaintiff is Elon Musk.


In [46]:
response = retrieval_chain.invoke({"input": "What was Mr. Page's response to Mr. Musk's concerns about AI, and how did it differ from Mr. Musk's viewpoint?"})

In [47]:
print(response["answer"])

Mr. Page responded to Mr. Musk's concerns about AI by stating that the potential replacement of humans by artificial intelligence systems would merely be "the next stage of evolution." He claimed that Mr. Musk was being a "specist" for favoring the human species over intelligent machines. This differed from Mr. Musk's viewpoint, as he expressed his concern about AI potentially surpassing human intelligence and posing a threat to humanity.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [48]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [49]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [50]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [51]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What strategy video game did OpenAI compete in?,"OpenAI competed in Dota 2, a strategy video ga...",[77. \nInitial work at OpenAI followed much in...,"OpenAI competed in Dota 2, a strategy video game.",1.000000,1.000000,1.0,1.000000,0.741034
1,How did OpenAI use reinforcement learning in t...,OpenAI used reinforcement learning to compete ...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to play Dot...,0.666667,0.872291,0.5,1.000000,0.540233
2,How would IP assets be used to generate profit...,"If the proposed business model is implemented,...","[business model were valid, it would radically...",nan,1.000000,0.888121,0.0,0.000000,0.179205
3,Which strategy video game did OpenAI beat a wo...,OpenAI beat a world-champion program in Dota 2...,[77. \nInitial work at OpenAI followed much in...,OpenAI beat a world-champion program in the ga...,NaN,0.965201,1.0,1.000000,0.621299
4,"""What was the impact of OpenAI's models on fut...",OpenAI's models had a significant impact on fu...,"[challenging.” At the time, OpenAI stated that...",Their publication did prove to be useful to th...,1.000000,0.939927,1.0,1.000000,0.808500
5,"""What game did OpenAI use reinforcement learni...",OpenAI used reinforcement learning to compete ...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to compete ...,1.000000,0.932460,1.0,1.000000,0.740496
6,What was Mr. Page's response to Mr. Musk's con...,Mr. Page responded to Mr. Musk's concerns abou...,"[Page, then-CEO of Google’s parent company Alp...",Mr. Page responded that would merely “be the n...,1.000000,0.973205,1.0,0.916667,0.528703
7,What are the US officials' concerns about UAE'...,The US officials are concerned about the Unite...,[chip fabrication plants. If $10 billion from ...,US officials are concerned about UAE's ties to...,1.000000,0.964566,1.0,0.700000,0.709427
8,What were Stephen Hawking's concerns about AGI...,"Stephen Hawking, along with Elon Musk and Bill...",[18. \nMr. Musk has long recognized that AGI p...,Stephen Hawking's concerns about AGI in the wr...,0.000000,0.926072,1.0,1.000000,0.482369
9,What model did OpenAI release in January 2018 ...,OpenAI released the Generative Pre-Trained Tra...,[those connections to the target language. \n7...,OpenAI released the Generative Pre-Trained Tra...,1.000000,0.886415,1.0,0.750000,0.890677


## Task 4: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [55]:
results

{'faithfulness': 0.9667, 'answer_relevancy': 0.9356, 'context_recall': 0.8500, 'context_precision': 0.8583, 'answer_correctness': 0.7326}

And see how our advanced retrieval modified our chain!

In [56]:
advanced_retrieval_results

{'faithfulness': 0.8519, 'answer_relevancy': 0.9348, 'context_recall': 0.8500, 'context_precision': 0.8367, 'answer_correctness': 0.6242}

In [57]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.966667,0.851852,-0.114815
1,answer_relevancy,0.935556,0.934826,-0.000730
2,context_recall,0.850000,0.850000,0.000000
3,context_precision,0.858333,0.836667,-0.021667
4,answer_correctness,0.732586,0.624194,-0.108391


## Task 5: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [58]:
# load a new embedding model 
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [59]:
# new faiss based vector store using the documents and new embedding model 
vector_store = FAISS.from_documents(documents, new_embeddings)

In [60]:
# intialize new retriever 
new_retriever = vector_store.as_retriever()

In [61]:
# Initialize from llm using default template.
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [62]:
# Create retrieval chain that retrieves documents and then passes them on.
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [63]:
# run the test questions against the new retrieval chain
answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [64]:
# create a new dataset with the results
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [65]:
# evaluate the new results
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [66]:
# print the results
new_advanced_retrieval_results

{'faithfulness': 0.9333, 'answer_relevancy': 0.9335, 'context_recall': 0.8500, 'context_precision': 0.8611, 'answer_correctness': 0.6791}

In [67]:
# create comparaisons between models

df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'Text Embedding 3'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['Delta - TE3 -> ADA'] = df_merged['Text Embedding 3'] - df_merged['ADA']
df_merged['Delta - TE3 -> Baseline'] = df_merged['Text Embedding 3'] - df_merged['Baseline']

df_merged

,Metric,Baseline,ADA,Text Embedding 3,Delta - TE3 -> ADA,Delta - TE3 -> Baseline
0,faithfulness,0.966667,0.851852,0.933333,0.081481,-0.033333
1,answer_relevancy,0.935556,0.934826,0.933540,-0.001285,-0.002016
2,context_recall,0.850000,0.850000,0.850000,0.000000,0.000000
3,context_precision,0.858333,0.836667,0.861111,0.024444,0.002778
4,answer_correctness,0.732586,0.624194,0.679133,0.054939,-0.053452


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

> Answer:
> It appears that TE3 foes marginally better than ADA for this set of content. Doing slighly worse for answer_relavancy. 

## BONUS ACTIVITY: Showcase Multi-Context Perfomance Changes

Now that we've looked at a number of different examples - showcase the difference on the multi-context *specific* questions that were synthetically generated.

> NOTE: You have all the data you'll need already in the notebook if you made it to this step!

In [88]:
question = "What were Stephen Hawking's concerns about AGI in the wrong hands?"

result_baseline = retrieval_augmented_qa_chain.invoke({"question" : question})
result_original = retrieval_chain.invoke({"input" : question})
result_comparison = new_retrieval_chain.invoke({"input" : question})

print(result_baseline['response'].content)
print(result_original['answer'])
print(result_comparison['answer'])

Stephen Hawking's concerns about AGI in the wrong hands were that "the future doesn't need us."
Stephen Hawking, along with other luminaries like Bill Joy, raised concerns about AGI falling into the wrong hands. They emphasized that if a machine can solve tasks better than humans, it becomes more economically useful, potentially leading to a future where "the future doesn’t need us." Hawking saw AGI as "probably the greatest threat to the continued existence of humanity" and believed that it should be fought against.
Stephen Hawking, along with luminaries like Bill Joy and Elon Musk, raised concerns about AGI posing a grave threat to humanity. The concern was that if a machine could solve nearly any task better than humans, it would become more economically useful than humans, potentially leading to a future where "the future doesn’t need us." This implies that AGI in the wrong hands could lead to a scenario where humans are no longer essential or needed in society.
